# MCF Instances

## Formulations of MCNF Problems

Source: [Input Format for Multicommodity Flow Problems](ftp://ftp.cs.wisc.edu/math-prog/cpo-dataset/networks/multicommodity/Jones_Lustig_Farv/format.tex)

The JL format for MCNF problems defines these different ways to formulate a problem:
- **origin-destination problem (ODP)** - The commodity is defined as a product $k$, that travels between a specific *origin* $s$ and a specifig *destination* $t$. The commodity is then a triple $$\kappa = (k, s, t)$$
- **destination specific problem (DSP)** - The commodity is defined as a product that travels to a specific destination from multiple origins, or vice versa, from a specific origin to multiple destinations. The commodity is then a tuple $$\kappa = (k, t) \quad \text{or} \quad \kappa = (k, s)$$
- **product specific problem (PSP)** - The commodity is defined as a product that must travel through the network from multiple origins to multiple destinations. The commodity is then the singleton $$\kappa = k$$.

Problem instances for the Linear Multicomodity Flow Problems are taken from
[Multicommodity Problem Instances](http://groups.di.unipi.it/optimize/Data/MMCF.html#MNetGen)

In [13]:
import numpy as np

In [3]:
from pathlib import Path

# Path to data directory
instances_path = Path("example/")

assert instances_path.exists()

## MNETGEN

### Input Format Overview

A problem instance consist of 4 files:
- **Node file** (``*.nod``) for general information
- **Arc file** (``*.arc``) describing links
- **Supply/demand file** (``*.sup``)
- **Mutual capacity file** (``*.mut``)

#### Node File

The node file contains four integers int the following order:
```
number of products
number of nodes
number of links
number of bundled links
```

In [4]:
# Path to the node file
nod_file = instances_path / "mnetgen.nod"

In [5]:
content = None
with open(nod_file, 'r') as fin:
    content = fin.read()
    
products_no, nodes_no, links_no, bundled_links_no = map(int, content.split())

print(products_no, nodes_no, links_no, bundled_links_no)

4 64 196 84


#### Link File

One line of the link file contains numbers separated by whitespace.
The meaning of the numbers for a given links is specified in the following order:

```
arcname fromnode tonode commodity cost individual_capacity mutual_capacity_ptr
```

Arc name is an integer between 1 and the number of arcs (differently from the original mnetgen format), that is necessary to distinguish between multiple instances of an arc (i, j) for the same commodity, that are permitted.

The mutual capacity pointer is an ``integer`` specifying the instace of the bundled link found in the *mutual capacity file*.
*NOTE: **0** means no pointer is specified.*

The number ``-1`` in a column is used to specify that:
    - the information in the column is to be replicated over all possible instances
    - the information is not applicable to the problem
The meaning is derived from the context.

In mnetgen case ``-1`` means:
    - no individual capacity in the ``individual_capacity`` field, meaning the amount of product is bounded only by the upper bound

In [6]:
# Path to the link file
arc_file = instances_path / "mnetgen.arc"

In [96]:
from dataclasses import dataclass

@dataclass
class ArcsInfo:
    """ Information about links. """
    
    arcnames : np.ndarray
    fromnodes : np.ndarray
    tonodes : np.ndarray
    commodities : np.ndarray
    costs : np.ndarray
    individual_capacities : np.ndarray
    mutual_ptrs : np.ndarray
    

def read_arc(arc_file):
    arcnames, fromnodes, tonodes, commodities, costs, individual_capacities, mutual_ptrs = [], [], [], [], [], [], []

    with open(arc_file, 'r') as fin:
        for line in fin:
            arcname, fromnode, tonode, commodity, cost, icapacity, mptr = line.split()
       
            arcnames.append(int(arcname))
            fromnodes.append(int(fromnode))
            tonodes.append(int(tonode))
            commodities.append(int(commodity))
            costs.append(float(cost))
            individual_capacities.append(float(icapacity))
            mutual_ptrs.append(int(mptr))
        
    arcnames = np.array(arcnames, dtype='uint32')
    fromnodes = np.array(fromnodes, dtype='int32')
    tonodes = np.array(tonodes, dtype='int32')
    commodities = np.array(commodities, dtype='int32')
    costs = np.array(costs, dtype='float')
    individual_capacities = np.array(individual_capacities, dtype='int32')
    mutual_ptrs = np.array(mutual_ptrs, dtype='uint32')

    return ArcsInfo(arcnames = arcnames,
                    fromnodes = fromnodes,
                    tonodes = tonodes,
                    commodities = commodities,
                    costs = costs,
                    individual_capacities = individual_capacities,
                    mutual_ptrs = mutual_ptrs)

arcinfo = read_arc(arc_file)

mask = arcinfo.arcnames == 10
print(arcinfo.fromnodes[mask])
print(arcinfo.tonodes[mask])
print(arcinfo.commodities[mask])
print(arcinfo.costs[mask])
print(arcinfo.individual_capacities[mask])
print(arcinfo.mutual_ptrs[mask])

[22 22 22 22]
[39 39 39 39]
[1 2 3 4]
[100.    97.68  21.88  84.9 ]
[-1 32 -1 -1]
[9 9 9 9]


#### Supply/Demand file

One line of the link file contains numbers separated by whitespace.

```
node product supply
```

In [8]:
# Path to the link file
sup_file = instances_path / "mnetgen.sup"

In [9]:
supply = []
with open(sup_file, 'r') as fin:
    for line in fin:
        supply.append(line.split())
        
print(*supply[:20], sep='\n')

['1', '1', '32']
['1', '2', '32']
['1', '3', '32']
['1', '4', '32']
['2', '1', '32']
['2', '2', '32']
['2', '3', '32']
['2', '4', '66']
['3', '1', '32']
['3', '2', '32']
['3', '3', '32']
['3', '4', '32']
['4', '1', '32']
['4', '2', '32']
['4', '3', '32']
['4', '4', '32']
['5', '1', '32']
['5', '2', '32']
['5', '3', '32']
['5', '4', '32']


#### Mutual capacity file

One line of the link file contains numbers separated by whitespace.

```
mutual_capacity_ptr mutual_capacity
```

In [10]:
# Path to the link file
mut_file = instances_path / "mnetgen.mut"

In [99]:
mutual = []
with open(mut_file, 'r') as fin:
    for line in fin:
        nums = list(map(int, line.split()))
        mutual.append(nums)
        
mutual = np.array(mutual)

mutual

array([[  1, 128],
       [  2, 128],
       [  3, 162],
       [  4, 162],
       [  5, 162],
       [  6, 162],
       [  7, 162],
       [  8, 128],
       [  9, 128],
       [ 10, 128]])

## PDS

#### Node File

The node file contains four integers in the following order:
```
number of products
number of nodes
number of links
number of bundled links
```

### Link File

One line of the link file contains numbers separated by whitespace.
The meaning of the numbers for a given links is specified in the following order:

*Note: Different from MNETGEN*

```
fromnode tonode commodity cost individual_capacity mutual_capacity_ptr
```

#### Supply/Demand file

```
node product supply/demand
```

Supply is expressed as a positive integer, demand as a negative integer


In [101]:
basename = "pds"
nod_file = instances_path / (basename + ".nod")
arc_file = instances_path / (basename + ".arc")
sup_file = instances_path / (basename + ".sup")
mut_file = instances_path / (basename + ".mut")